In [1]:
%load_ext autoreload
%autoreload 2

In [23]:
import numpy as np
from ipywidgets import HBox, VBox, Label, Layout, Button, Output
from IPython.display import display
from functools import partial
import random
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
import math
import copy
from enum import Enum
from abc import ABC, abstractmethod
from dataclasses import dataclass
from base import View, Agent, Mark, Action
from Game import Game
from QLearningApproximationAgent import QLearningApproximationAgent
from EpsilonGreedyPolicy import EpsilonGreedyPolicy

In [26]:
class JupyterView(View):
    def __init__(self, update_on_end: bool = False):
        self.update_on_end = update_on_end
        self.out = Output()
    
    def on_click(self, row, column, button):
        self.disable_all_buttons()
        self.turn_callback(Action(row, column))
    
    def update(self, state):
        if self.update_on_end and state.is_game_end:
            label = Label()
            if state.is_game_end:
                if state.winner == Mark.NO:
                    label.value = "Draw!"
                else:
                    label.value = state.winner.name + ' wins!'
            else:
                label.value = state.current_player.name + "'s turn"

            h, w = state.board.shape
            self.buttons = []
            buttons = []

            for r in range(h):
                line = []
                for c in range(w):
                    button = Button(
                        description='',
                        layout=Layout(width='20px', height='20px', padding='0px')
                    )

                    button.on_click(partial(self.on_click, r, c))

                    if state.board[r][c] == Mark.X.value:
                        button.description = Mark.X.name
                        button.disabled = True
                    elif state.board[r][c] == Mark.O.value:
                        button.description = Mark.O.name
                        button.disabled = True
                    elif state.is_game_end:
                        button.disabled = True

                    line.append(button)
                    self.buttons.append(button)

                buttons.append(HBox(line))

            self.out.clear_output()
            with self.out:
                display(VBox([label, VBox(buttons)]))
        
    def disable_all_buttons(self):
        for b in self.buttons:
            b.disabled = True
    


In [16]:
class JupyterHumanAgent(Agent):
    def __init__(self):
        pass
    
    def get_action(self, state):
        return None
    
    def win(self, state):
        pass
    
    def loss(self, state):
        pass
    
    def draw(self, state):
        pass

In [17]:
g = Game(player1=JupyterHumanAgent(), player2=JupyterHumanAgent(), view=JupyterView())
g.view.out

Output()

In [28]:
max_row = 5
num_feat = ((((max_row - 1) * 2) - 1) * 2) + 1
theta = np.zeros(num_feat)
agent1 = QLearningApproximationAgent(0.5, 0.5, EpsilonGreedyPolicy(0.05), theta=theta)
agent2 = QLearningApproximationAgent(0.5, 0.5, EpsilonGreedyPolicy(0.05), theta=theta)
g = Game(player1=agent1, player2=agent2, view=JupyterView(update_on_end=True))
g.view.out

Output()

In [29]:
theta

array([ -1. ,  -1. ,  -1. ,  50. ,   0. ,   0. ,   0. ,  -1. ,  -1. ,
       -50.5,   0. ,   0. ,   0. ,   0. ,  -1.5])

In [34]:
max_row = 5
num_feat = ((((max_row - 1) * 2) - 1) * 2) + 1
theta = np.zeros(num_feat)
for i in tqdm(range(10)):
    print(f'Game {i}')
    print(f'Theta {theta}')
    agent1 = QLearningApproximationAgent(0.5, 0.5, EpsilonGreedyPolicy(0.05), theta=theta, inf_field=False)
    agent2 = QLearningApproximationAgent(0.5, 0.5, EpsilonGreedyPolicy(0.05), theta=theta, inf_field=False)
    g = Game(player1=agent1, player2=agent2, view=None, m=20, n=20)


Game 0
Theta [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Game 1
Theta [-50.109375   -1.5        -2.7265625  50.75        0.          0.
 -48.609375   49.25       -1.5       -50.8359375   0.          0.
   0.         50.75       -1.5      ]
Game 2
Theta [1.24344545e+09 8.30387477e+07 2.20796003e+08 5.44363915e+08
 3.71890811e+08 9.17256985e+08 6.10129422e+08 5.74331541e+08
 1.86952749e+08 6.64004470e+08 0.00000000e+00 9.38860226e+08
 3.61298781e+08 2.41823203e+08 1.13125000e+01]


/mnt/e/Нетология/Diploma/m,n,k-game/QLearningApproximationAgent.py:227: RuntimeWarning: invalid value encountered in multiply
  self.theta += self.alfa * (reward + future_reward - prev_action.new_Q_value) * prev_action.new_features


IndexError: Cannot choose from an empty sequence

In [ ]:
weights

### TODO:
* UI переделать на jp_proxy_widget
* Выделить апроксимирующую модель в отдельный класс